In [1]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [2]:
import json
import openai_helper
from azure.search.documents import SearchClient
import search_helper_alta
import cosmosdb_helper
#output_file_name = "alta_enriched_data.json"
#vector_file_name = f"{output_file_name}_with_vectors.json"
#with open(vector_file_name, "r") as f:
#    aml_index_data_with_vectors = json.loads(f.read())
#print(len(aml_index_data_with_vectors))


index_name = "alta_index"

Azure Search EndPoint: https://anildwaaisearch.search.windows.net
Index Name: aml_index_2
Azure OpenAI Endpoint: https://anildwaapim.azure-api.net
Azure OpenAI Large Embedding Deployment: text-embedding-ada-002
Azure OpenAI Small Embedding Deployment: text-embedding-ada-002
Embedding Model Name: text-embedding-ada-002
Azure OpenAI API Version: 2024-06-01
Azure Document Intelligence Endpoint: https://anildwadocintelliwestus2.cognitiveservices.azure.com/
CosmosDB endpoint:  https://anildwacosmoswestus.documents.azure.com:443/
database_name:  vectordb
container_name:  vectortest_hybridsearch_2partitions


v14758586 -

v14758723 -

v14758175 - 

v16141620

v16144611 -

id-SF0A0166207

v15493450 -

v15492295 -

In [3]:
#aml_index_data_with_vectors[0]

In [4]:

async def rephrase_user_question(user_question, top_k=10):

    search_rephrase_system_message = """
    You are a search assistant. You need to break up the user question into smaller questions such that
    each question can be used to retrieve context from search engine. 
    The question needs to be broken logically without losing the original intent of the question.
    Provide the broken down questions in the following json format:

    Do not include ``` or ```json in the response. Just provide the json object.
    {
        "original_question": "original question",
        "sub_questions": [
            "sub question 1",
            "sub question 2",
            
        ]
        
    }


    """

    #user_question = "what are the steps for installing netbackup for DB2 and explain each step in detail?"

    llm_response = openai_helper.getOpenAIResp(userQuery=user_question, systemMessage=search_rephrase_system_message)

    user_questions = json.loads(llm_response)["sub_questions"]

    print(f"User Questions = {user_questions}")
    complete_context = ""

    for q in user_questions:
        sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=q, top_k=top_k)
        complete_context += json.dumps(sr)


    print(f"Context Token Count1 = {openai_helper.get_token_count(complete_context)}")

    return complete_context

In [16]:
def get_llm_response(user_query, context):
    system_message = f"""
    You are a Veritas Backup product Search assistant. 
    You only need to answer questions related to Veritas Backup products and using the context provided below and nothing else. 
    
    Answer the user question completely and accurately without omitting any information from only the Context provided. 
    
    If the question cannot be answered using the context provided, you can state that you cannot answer the question.

    <Context>{context}</Context>

    Provide the sources of your answers using the format below:
    [section: section_id]

    Provide detailed and accurate answers to the user question from only the Context provided.
    """
    #print(context)
    print(f"Context Token Count = {openai_helper.get_token_count(context)}")

    escaped_system_message = system_message.replace('\\', '\\\\').replace('"', '\\"').replace('\n', '\\n')
    escaped_user_query = user_query.replace('\\', '\\\\').replace('"', '\\"').replace('\n', '\\n')

    print(f"system_message = {escaped_system_message}")
    print(f"user_query = {escaped_user_query}")
    llm_response = openai_helper.getOpenAIResp(userQuery=user_query, systemMessage=system_message)
    print(llm_response)

In [19]:
search_query = "how to enable a policy to archive the logs in DB2 and give me more information on user archive schedules?"

#context = await rephrase_user_question(search_query)
sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=search_query, top_k=20)
context = f"<Context>{ json.dumps(sr)} </Context>"
get_llm_response(user_query=search_query, context=context)

Context Token Count = 17522
system_message = \n    You are a Veritas Backup product Search assistant. \n    You only need to answer questions related to Veritas Backup products and using the context provided below and nothing else. \n    \n    Answer the user question completely and accurately without omitting any information from only the Context provided. \n    \n    If the question cannot be answered using the context provided, you can state that you cannot answer the question.\n\n    <Context><Context>[[{\"id\": \"3ce599f8-b484-40ff-a56a-2bfa613dd39e\"}, {\"part_title\": \"\"}, {\"chapter_title\": \"Configuring NetBackup for DB2\"}, {\"section_title\": \"Configuring a policy to archive the archive logs\"}, {\"para\": \"This topic describes how to create a policy to archive the NetBackup DB2 archive log entries in the ARCDIR directory. Follow these instructions if you want to use the user exit program with the ARCFUNC COPY command.When NetBackup performs an archive, it deletes the o

In [20]:
#%%skip
search_query = "what are the steps for installing netbackup for DB2 and explain each step in detail?"

#context = await rephrase_user_question(search_query)
sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=search_query, top_k=20)
context = f"<Context>{ json.dumps(sr)} </Context>"
get_llm_response(user_query=search_query, context=context)

Context Token Count = 11678
system_message = \n    You are a Veritas Backup product Search assistant. \n    You only need to answer questions related to Veritas Backup products and using the context provided below and nothing else. \n    \n    Answer the user question completely and accurately without omitting any information from only the Context provided. \n    \n    If the question cannot be answered using the context provided, you can state that you cannot answer the question.\n\n    <Context><Context>[[{\"id\": \"3cec87d5-df82-4f30-b90b-822de8d09a67\"}, {\"part_title\": \"\"}, {\"chapter_title\": \"Installing NetBackup for DB2\"}, {\"section_title\": \"Planning the installation of NetBackup for DB2 \"}, {\"para\": \"\\n\\t\\t\\t\\t shows the major installation steps that are needed to run NetBackup for DB2. Each step contains one or more links to pertinent procedures and concepts.StepActionDescriptionStep 1Verify the installation prerequisites. \\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\n\\t

In [ ]:
#%%skip
search_query = "how to enable a policy to archive the logs in DB2 and give me more information on user archive schedules?"
context = await rephrase_user_question(search_query)
#sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=search_query, top_k=20)
#context = f"<Context>{ json.dumps(sr)} </Context>"
get_llm_response(user_query=search_query, context=context)

Using Azure OpenAI: https://anildwaopenaiwestus.openai.azure.com//openai/
[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}]
User Questions = ['How to enable a policy to archive the logs in DB2?', 'What are the steps to configure log archiving in DB2?', 'What are user archive schedules in DB2?', 'How to manage and configure user archive schedules in DB2?']
Context Token Count1 = 16112


In [40]:
#%%skip
search_query = "how to configure a policy to archive the archive logs in DB2 and what is the additional configuration needed if netbackup installed on more than one node in the DB2 cluster?"
context = await rephrase_user_question(search_query, top_k=20)
#sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=search_query, top_k=20)
#context = f"<Context>{ json.dumps(sr)} </Context>"
#get_llm_response(user_query=search_query, context=context)


Using Azure OpenAI: https://anildwaopenaiwestus.openai.azure.com//openai/
[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}]
User Questions = ['How to configure a policy to archive the archive logs in DB2?', 'What additional configuration is needed if NetBackup is installed on more than one node in the DB2 cluster?']
Context Token Count1 = 19745


#### Cosmos DB Vector Search

In [42]:
llm_prompt = "Explain each step in detail based on sections and references from the context provided.\n"
search_query = "what are the steps for installing netbackup for DB2 and explain each step in detail?"
ctx = cosmosdb_helper.get_cosmosdb_context(search_query, 10)
context = f"<Context>{ctx} </Context>"
get_llm_response(user_query=llm_prompt + search_query, context=context)

Context Token Count = 18622
Using Azure OpenAI: https://anildwaopenaiwestus.openai.azure.com//openai/
[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}]
The steps for installing NetBackup for DB2 are outlined in the context provided. Here is a detailed explanation of each step:

1. **Verify the installation prerequisites**:
   - Before starting the installation, ensure that all prerequisites are met. This includes checking the compatibility of your operating system and platform with NetBackup for DB2. You can verify this by visiting the NetBackup compatibility list site at [NetBackup Compatibility](http://www.netbackup.com/compatibility). This step ensures that your environment is suitable for the installation and operation of NetB

In [44]:
search_query = "how to enable a policy to archive the logs in DB2 and give me more information on user archive schedules?"
ctx = cosmosdb_helper.get_cosmosdb_context(search_query, 10)
context = f"<Context>{ctx} </Context>"
get_llm_response(user_query=llm_prompt + search_query, context=context)

Context Token Count = 14439
Using Azure OpenAI: https://anildwaopenaiwestus.openai.azure.com//openai/
[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}]
To enable a policy to archive the logs in DB2 and to provide more information on user archive schedules, follow these detailed steps:

### Step-by-Step Process to Enable a Policy to Archive Logs in DB2

1. **Determine the Method for Log Archiving**:
   - The policy you use to back up the archive logs depends on the method you use for log archiving.
   - If you use the user exit program, create a Standard policy.
   - If you use the VENDOR method, you can use the DB2 Application Backup schedule.
   [section: v24410043]

2. **Create a New Policy**:
   - Start the NetBackup web UI.
  

In [46]:
search_query = "how to configure a policy to archive the archive logs in DB2 and what is the additional configuration needed if netbackup installed on more than one node in the DB2 cluster?"
ctx = cosmosdb_helper.get_cosmosdb_context(search_query, 10)
context = f"<Context>{ctx} </Context>"
get_llm_response(user_query=llm_prompt + search_query, context=context)

Context Token Count = 18761
Using Azure OpenAI: https://anildwaopenaiwestus.openai.azure.com//openai/
[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}]
To configure a policy to archive the archive logs in DB2 and the additional configuration needed if NetBackup is installed on more than one node in the DB2 cluster, follow these detailed steps:

### Configuring a Policy to Archive the Archive Logs

1. **Start the NetBackup Web UI**:
   - Open the NetBackup web UI to begin the configuration process.

2. **Create a New Policy**:
   - For Windows, create a new MS-Windows policy type.
   - For UNIX, create a new Standard policy type.
   - Specify the attributes for the policy.

3. **Create a User Archive Schedule**:
   - On the Schedul